Ensure Dependencies:

In [5]:
pip install torch torchvision pillow numpy requests

Defaulting to user installation because normal site-packages is not writeable
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached idna-3.11-py3-none-any.whl (71 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)

   ---------------------------------------- 0/4 [urllib3]
   ---------------------------------------- 0/4 [urllib3]
   ---------------------------------------- 0/4 [urllib3]
   ---------------------------------------- 0/4 [urllib3]
   ---------- ----------------------------- 1/4 [idna]
   -------------------- ------------------- 2/4 [charset_normalizer]
   -------------------- ------------------- 2/4 [charset_normalizer]
   ------------------------------ --------- 3/4 [requests]
   ------------------------------ --------- 3/4 [requests]
   ------------------------------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import torch
import torchvision
from torchvision import transforms, models
import torch.nn as nn
from PIL import Image
import json
import numpy as np
import os

# --- CONFIGURATION ---
# Add all your custom .pth files here. Give them a friendly name.
custom_models_to_load = {
    "Trained Using iPhone Photos": "smartstop_mobilenet_v2_iphoneimages.pth",
    "First Attempt ESP32-CAM": "smartstop_mobilenet_v2_esp32cam.pth",
    # "My Custom V2": "smartstop_v2_50epochs.pth", # Add more here later
}
CLASS_NAMES_FILE = 'class_names.json'
TEST_IMAGES = ["test1.jpg", "test2.jpg", "test3.jpg", "test4.jpg", "test5.jpg", "test6.jpg", "test7.jpg", "test8.jpg", "test9.jpg"]  # List of test images

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 1. RE-DEFINE YOUR TRANSFORMS ---
# Must use the EXACT same transforms as training
class SquarePad:
    def __call__(self, image):
        w, h = image.size
        max_wh = max(w, h)
        hp = int((max_wh - w) / 2)
        vp = int((max_wh - h) / 2)
        padding = (hp, vp, hp, vp)
        return transforms.functional.pad(image, padding, 0, 'constant')

inference_transform = transforms.Compose([
    SquarePad(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# A. Load Custom Class Names
with open(CLASS_NAMES_FILE, 'r') as f:
    custom_classes = json.load(f)
print(f"Loaded custom classes: {custom_classes}")

# B. Load Original ImageNet Labels (for the base model)
# We try to fetch them online for a readable output. Fallback to indices if offline.
imagenet_classes = None
try:
    # This is a standard list of the 1000 ImageNet classes
    import requests
    url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
    imagenet_classes = requests.get(url).text.split('\n')
    print("Successfully loaded ImageNet labels.")
except:
    print("Could not load ImageNet labels (offline?). Original model will show indices only.")

# ==========================================
# 3. MODEL LOADING HELPER FUNCTIONS
# ==========================================
loaded_models = {}

def load_original_model():
    """Loads the standard, pre-trained MobileNetV2"""
    print("Loading original ImageNet MobileNetV2...")
    model = models.mobilenet_v2(weights='DEFAULT')
    model.to(device)
    model.eval()
    return model

def load_custom_model(pth_path, num_classes):
    """Loads a fine-tuned model with your custom head"""
    if not os.path.exists(pth_path):
        print(f" [WARN] Model file not found: {pth_path}")
        return None
        
    print(f"Loading custom model from {pth_path}...")
    model = models.mobilenet_v2(weights=None) # No need for defaults, we have weights
    # Rebuild your custom head
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2),
        nn.Linear(model.last_channel, num_classes)
    )
    # Load your trained weights
    model.load_state_dict(torch.load(pth_path, map_location=device))
    model.to(device)
    model.eval()
    return model

# ==========================================
# 4. INITIALIZE ALL MODELS
# ==========================================
# 1. Load Original Base Model
loaded_models['Original (ImageNet)'] = {
    'model': load_original_model(),
    'classes': imagenet_classes,
    'type': 'imagenet'
}

# 2. Load All Custom Models
for name, path in custom_models_to_load.items():
    model = load_custom_model(path, len(custom_classes))
    if model:
        loaded_models[name] = {
            'model': model,
            'classes': custom_classes,
            'type': 'custom'
        }

print(f"\n--- Ready! Loaded {len(loaded_models)} models. ---")

# ==========================================
# 5. PREDICTION FUNCTION
# ==========================================
def predict_all(image_path):
    # 1. Load and Preprocess Image ONCE
    try:
        img = Image.open(image_path).convert('RGB')
        input_tensor = inference_transform(img)
        input_batch = input_tensor.unsqueeze(0).to(device)
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return

    print(f"\n====== Predictions for {image_path} ======")

    # 2. Loop through all loaded models
    for model_name, config in loaded_models.items():
        model = config['model']
        classes = config['classes']
        
        with torch.no_grad():
             output = model(input_batch)

        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        
        # --- Output Formatting ---
        print(f"\n[{model_name}]")
        
        if config['type'] == 'imagenet':
            # For the original model, show Top 3 guesses because it has 1000 classes
            top_probs, top_ids = torch.topk(probabilities, 3)
            for i in range(3):
                label = classes[top_ids[i]] if classes else str(top_ids[i].item())
                print(f"  #{i+1}: {label} ({top_probs[i].item()*100:.2f}%)")
                
        else:
            # For custom models, show the Top 1 guess
            top_prob, top_id = torch.max(probabilities, 0)
            label = classes[top_id.item()]
            print(f"  Result: {label} ({top_prob.item()*100:.1f}%)")
            # Optional: Show all custom class probabilities for debugging
            # for idx, prob in enumerate(probabilities):
            #      print(f"  - {classes[idx]}: {prob.item()*100:.1f}%")


# ==========================================
# 6. RUN TEST BATCH
# ==========================================
print(f"Starting batch prediction on {len(TEST_IMAGES)} images...")

for image_file in TEST_IMAGES:
    if os.path.exists(image_file):
        predict_all(image_file)
    else:
        print(f"\n [ERROR] Image not found: {image_file}")

Using device: cpu
Loaded custom classes: ['0_empty', '1_low', '2_medium', '3_high']
Successfully loaded ImageNet labels.
Loading original ImageNet MobileNetV2...
Loading custom model from smartstop_mobilenet_v2_iphoneimages.pth...
Loading custom model from smartstop_mobilenet_v2_esp32cam.pth...

--- Ready! Loaded 3 models. ---
Starting batch prediction on 9 images...

====== Predictions for test1.jpg ======

[Original (ImageNet)]
  #1: bolo tie (3.73%)
  #2: cleaver (2.77%)
  #3: hook (1.82%)

[Trained Using iPhone Photos]
  Result: 0_empty (72.0%)

[First Attempt ESP32-CAM]
  Result: 0_empty (63.6%)

====== Predictions for test2.jpg ======

[Original (ImageNet)]
  #1: tray (3.68%)
  #2: chest (3.65%)
  #3: spider web (2.66%)

[Trained Using iPhone Photos]
  Result: 0_empty (94.2%)

[First Attempt ESP32-CAM]
  Result: 1_low (40.8%)

====== Predictions for test3.jpg ======

[Original (ImageNet)]
  #1: lampshade (2.84%)
  #2: window shade (2.81%)
  #3: table lamp (1.73%)

[Trained Using 